# Packages


In [ ]:
!pip install seaborn
!pip install causalgraphicalmodels
!pip install matplotlib
!pip install -U scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import causalgraphicalmodels
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import tensorflow as tf

#  MSBD model building


In [ ]:
def sigmoid(x):
    return (1 / (1 + math.exp(-x)))
def epsilon_z1():
    return np.random.normal(loc = 1,scale = 0.5)
def epsilon_x1():
    return np.random.normal(loc = 0,scale = 1)
def epsilon_y1():
    return np.random.normal(loc = 0,scale = 1)
def epsilon_z2():
    return np.random.normal(loc = 0,scale = 0.5)
def epsilon_x2():
    return np.random.normal(loc = 0,scale = 1)
def epsilon_y2():
    return np.random.normal(loc = 0,scale = 0.5)
def para_c_a1():
    return np.random.normal(loc=2,scale=1)
def para_c_a2():
    return np.random.normal(loc=-2,scale=1)
def para_c_x1():
    return np.random.normal(loc=-1,scale=0.5)
def para_c_y1():
    return np.random.normal(loc=1,scale=0.8)
def para_c_b1():
    return np.random.normal(loc=1,scale=0.5)
def para_c_b2():
    return np.random.normal(loc=-1,scale=1)
def para_cxb1():
    return np.random.normal(loc=0.3,scale=1)
def para_cxb2():
    return np.random.normal(loc=0.2,scale=1)
def para_cyb1():
    return np.random.normal(loc=0.3,scale=1)
def para_cyb2():
    return np.random.normal(loc = -0.5,scale=1)


In [ ]:

def build_z1():
    return sigmoid(para_c_a1() + para_c_a2() *epsilon_z1() + epsilon_z1())
def build_x1(z_sum):
    return sigmoid(0.2*z_sum + epsilon_x1())
def build_y1(z1,x1,d):
    sum = 0.0
    for i in range(d):
        sum += z1[i] * para_c_y1()
    return sigmoid(sum - (2*x1 - 1) - epsilon_y1() - 2)
def build_x2(z1,x1,y1,z2,d):
    sum = 0.0
    sum_1 = 0.0
    for i in range(d):
        sum += z1[i]*para_cxb1()
        sum_1 += z2[i]*para_cxb2()
    return sigmoid(-sum + sum_1 + 2*x1 - 2 +y1 +epsilon_x2())
def build_y2(z1,x1,y1,z2,x2,d):
    sum = 0.0
    sum_1 = 0.0
    for i in range(d):
        sum += z1[i]*para_cyb1()
        sum_1 += z2[i]*para_cyb2()
    return sigmoid(-0.5*(sum + sum_1 + epsilon_y2() - 1) + 2*x1 + 2*x2-2 -epsilon_y2()+y1 -epsilon_y2())


In [ ]:
def build_causal_model(N,d):
    scm_np = np.empty((N,2*d+4))
    col_name = list()
    sum = 0.0
    for i in range(N):
        z1_list = list()
        z2_list = list()
        for j in range(d):
            if(i == 0):
                col_name.append('z1,'+str(j))
            z_1 = np.random.binomial(n=1, p=build_z1())
            scm_np[i][j] = z_1
            sum += z_1 * para_c_x1()
            z1_list.append(z_1)
        if i==0:
            col_name.extend(['x1','y1'])
        x_1 = np.random.binomial(n=1, p=build_x1(sum))
        scm_np[i][d] = x_1
        y_1 = np.random.binomial(n=1,p=build_y1(z1_list,x_1,d))
        scm_np[i][d+1] = y_1
        sum = 0.0
        for j in range(0,d):
            if(i == 0):
                col_name.append('z2,'+str(j))
            z_2 = np.random.binomial(n=1, p = sigmoid(z1_list[j]*para_c_b1() + (2*x_1-1)+epsilon_z2()+y_1-para_c_b2()))
            scm_np[i][d+2+j] = z_2
            z2_list.append(z_2)
        if i == 0:
            col_name.extend(['x2','y2'])
        x_2 =np.random.binomial(n=1,p= build_x2(z1_list,x_1, y_1,z2_list,d))
        scm_np[i][2*d+2] = x_2
        y_2 = build_y2(z1_list,x_1,y_1,z2_list,x_2,d)
        scm_np[i][2*d+3] = y_2
    return scm_np,col_name



# Do Operator


In [ ]:
def causal_do(scm,new_x1,new_x2,d):
    scm_copy = scm.copy()
    scm_copy[:,d] = new_x1
    scm_copy[:,2*d+2] = new_x2
    size = scm_copy.shape[0]
    z1_list = list()
    z2_list = list()
    for i in range(size):
        z1_list = scm_copy[i,0:d].tolist()
        y_1 = np.random.binomial(n=1,p=build_y1(z1_list,new_x1[i],d))
        scm_copy[i][d+1] = y_1
        for j in range(0,d):
            z_2 = np.random.binomial(n=1, p = sigmoid(z1_list[j]*para_c_b1() + (2*new_x1[i]-1)+epsilon_z2()+y_1-para_c_b2()))
            scm_copy[i][d+2+j] = z_2
            z2_list.append(z_2)
        y_2 = build_y2(z1_list,new_x1[i],y_1,z2_list,new_x2[i],d)
        scm_copy[i][2*d+3] = y_2

    return scm_copy

def estimate_conditional_expectation1(df,d,l_1,l_2):
    scm_cols = np.size(df,axis=1)
    scm_rows = np.size(df,axis=0)
    # result_arr = []
    c = df[:,d] == l_1[0]
    d = df[:,d] == l_1[1]
    a = df[:,scm_cols-2] == l_2[0]
    b = df[:,scm_cols-2] == l_2[1]
    z_z = np.zeros(scm_rows)
    z_z = z_z+a+c == 2
    #print(e)
    o_z = np.zeros(scm_rows)
    o_z = o_z+d+b == 2
    y = df[:,scm_cols-1]

    p = np.mean(y[z_z])
    q = np.mean(y[o_z])
    return [p,q]

def ab_test1(scm, n , d):
    n_a = int(n / 2)
    n_b = n - n_a
    result_dic = dict()
    result_arr = []
    result_temp = []
    set_variable_1 = np.array([0]*n_a + [1]*n_b)
    j=0
    set_variable_2 = np.array([0]*n_a + [0]*n_b)
    scm = causal_do(scm,set_variable_1,set_variable_2,d)
    #print(string)
    result_arr = estimate_conditional_expectation1(scm,d,[0,1],[0,0])
    set_variable_1 = np.array([0]*n_a + [1]*n_b)
    j=0
    set_variable_2 = np.array([1]*n_a + [1]*n_b)
    scm = causal_do(scm,set_variable_1,set_variable_2,d)

    result_temp = estimate_conditional_expectation1(scm,d, [0,1],[1,1])
    result_arr.extend(result_temp)

    print(result_arr)
    return result_arr

# Weighted Generation


In [ ]:
def p_x(fd,s):
    df_copy = fd[s].copy()
    size = fd.shape[0]
    pw2 = np.zeros(2)
    prob_w2 = np.zeros(size)
    for i in [0,1]:
        nm1 = df_copy == i
        pw2[i] = nm1.mean()
    for i in range(size):
        if df_copy[i] == 0:
            prob_w2[i] = pw2[0]
        else :
            prob_w2[i] = pw2[1]
    return prob_w2

def p_x2_x1(fd):
    df_copy = fd.copy()
    proba_x2_x1 = np.ones(fd.shape[0])
    X = df_copy['x1'].values.reshape(-1,1)
    y = df_copy['x2']
    clf = LogisticRegression(random_state = 0).fit(X,y)
    predict = clf.predict_proba(X)
    for j in range(fd.shape[0]):
        if( y[j] == 0):
            proba_x2_x1[j] = predict[j,0]
        else :
            proba_x2_x1[j] = predict[j,1]
    return proba_x2_x1

def p_x1_z1(fd):
    df_copy = fd.copy()
    proba_x1_z1 = np.ones(fd.shape[0])
    z1 = df_copy.iloc[:,0:d]
    X = z1.values.reshape(-1,d)
    y = df_copy['x1']
    clf = LogisticRegression(random_state=0).fit(X, y)
    predict = clf.predict_proba(X)
    for j in range(fd.shape[0]):
        if( y[j] == 0):
            proba_x1_z1[j] = predict[j,0]
        else :
            proba_x1_z1[j] = predict[j,1]
    return proba_x1_z1

def p_x2_oth(fd):
    df_copy = fd.copy()
    y = df_copy['x2']
    df_copy = df_copy.drop(columns = ['x2','y2'])
    d = df_copy.shape[1]
    X = df_copy.values.reshape(-1,d)
    proba_x2 = np.ones(fd.shape[0])
    clf = LogisticRegression(random_state=0).fit(X, y)
    predict = clf.predict_proba(X)
    for j in range(fd.shape[0]):
        if( y[j] == 0):
            proba_x2[j] = predict[j,0]
        else :
            proba_x2[j] = predict[j,1]
    return proba_x2


def weightGenerator(df):
   data1 = np.multiply(p_x(df,'x1'),p_x(df,'x2'))
   data2 = np.multiply(p_x1_z1(df),p_x2_oth(df))
   data_prob = np.divide(data1,data2)


# CWO

In [ ]:

def cwoBeta(X,y,weight):
  Beta1 = LinearRegression().fit(X,y,sample_weight=weight)
  test_x=np.array([[0,0],[1,0],[0,1],[1,1]]).reshape((-1, 2))

  y_pred=Beta1.predict(test_x)

  return y_pred
def weightedATE(df):
    data = df.copy()
    X = data[['x1','x2']].values.reshape(-1,2)
    y = data['y2'].values.reshape(-1,1)
    weight = weightGenerator(df)
    ate = cwoBeta(X,y,weight)
    #print(ate)
    return ate

# Neural Network

In [ ]:
def nnBeta(X,y,testX,weight,hp):
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp['input_units'],activation='relu',input_shape=(2,)))
  Beta2.add(tf.keras.layers.Dropout(hp['dropout']))
  for i in range(hp['n_layers']):
    Beta2.add(tf.keras.layers.Dense(hp[f'conv_{i}_units'], activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp[f'dropout_{i}_']))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp['learning_rate']), loss='mse')

  early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=40, verbose=0, mode='auto', restore_best_weights=True)
  history=Beta2.fit(X,y,epochs=1000,
                    validation_split=0.2,
                    shuffle = True,
                    callbacks=[early_stop],
                    verbose=0
                    ,sample_weight=weight
                    )


  y_pred = Beta2.predict(testX)
  return y_pred

In [ ]:
def NeuralATE(df,hp):
  X = df[['x1','x2']].values.reshape(-1,2)
  y = df['y2'].values.reshape(-1,1)
  sampleWeight = weightGenerator(df)
  test=np.array([[0,0],[1,0],[0,1],[1,1]]).reshape((-1, 2))
  test_x=pd.DataFrame(test,columns=['a','b'])
  XX=test_x.values.reshape(-1, 2)
  ate = nnBeta(X,y,XX,sampleWeight,hp)


  return ate


# Result Storing in Pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
d=7

In [ ]:
def doDo(d):
  N=10000000
  fdDo,col_name=build_causal_model(N,d)
  print(fdDo)
  mu0,mu1,mu2,mu3=ab_test1(fdDo, N,d)
  df = pd.DataFrame(fdDo,columns = col_name)
  return df,mu0,mu1,mu2,mu3

In [ ]:
fd,mu0,mu1,mu2,mu3=doDo(d)
print("Estimated ATE: {:.3f}".format(mu1-mu0))
print("mu(1): {:.3f}".format(mu1))
print("mu(0): {:.3f}".format(mu0))

In [ ]:
fd.to_pickle("drive/MyDrive/Datasets/msbd_dataframe_7.pkl")

In [ ]:
string1 = str(mu0) + " " + str(mu1) + " " + str(mu2) + " " + str(mu3)
print(string1)
f = open("drive/MyDrive/Datasets/msbd_mu_5.txt","w")
f.write(string1)
f.close()

# Pickle data extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
d=8
fd = pd.read_pickle("drive/MyDrive/Datasets/msbd_dataframe_88.pkl")
print(fd)
f = open('drive/MyDrive/Datasets/msbd_mu_88.txt','r')
mm = f.read()
mu=np.ones(4)
mu0,mu1,mu2,mu3 = mm.split()
mu[1]=float(mu1)
mu[0]=float(mu0)
mu[2]=float(mu2)
mu[3]=float(mu3)
print(mu0,mu1,mu2,mu3)

# Result Generation


In [ ]:
def MAAEgenerator(fd,mu):

  muN=NeuralATE(fd,hpDict)

  muC=weightedATE(fd)
  maaeN=(abs(muN[0]-mu[0])+abs(muN[1]-mu[1])+abs(muN[2]-mu[2])+abs(muN[3]-mu[3]))/4
  maaeC=(abs(muC[0]-mu[0])+abs(muC[1]-mu[1])+abs(muC[2]-mu[2])+abs(muC[3]-mu[3]))/4
  return maaeN,maaeC

#print(MAAEgenerator(fd,mu1,mu0))

In [ ]:
for N_SAMPLE in range(500,501,500):
  muN=np.zeros(100)
  muC=np.zeros(100)
  for k in range(100):
    # print("K= ",k)
    db=fd.sample(n=N_SAMPLE).reset_index(drop=True)
    muN[k],muC[k]=MAAEgenerator(db,mu)
    # print(k,muN[k],muC[k])
  MAAE_N=np.median(muN)
  print("sample size" , N_SAMPLE)
  print("MAAE NN",MAAE_N)
  MAAE_C=np.median(muC)
  print("MAAE CWO",MAAE_C)
  print('loooooolllllll')

sample size 500
MAAE NN 0.03483150899410248
MAAE CWO 0.03419560898187281
loooooolllllll


In [ ]:
for N_SAMPLE in range(10000,10001,500):
  muN=np.zeros(100)
  muC=np.zeros(100)
  for k in range(100):
    # print("K= ",k)
    db=fd.sample(n=N_SAMPLE).reset_index(drop=True)
    muN[k],muC[k]=MAAEgenerator(db,mu)
    # print(k,muN[k],muC[k])
  MAAE_N=np.median(muN)
  print("sample size" , N_SAMPLE)
  print("MAAE NN",MAAE_N)
  MAAE_C=np.median(muC)
  print("MAAE CWO",MAAE_C)
  print('loooooolllllll')

In [ ]:
for N_SAMPLE in range(7500,10001,500):
  muN=np.zeros(100)
  muC=np.zeros(100)
  for k in range(100):
    # print("K= ",k)
    db=fd.sample(n=N_SAMPLE).reset_index(drop=True)
    muN[k],muC[k]=MAAEgenerator(db,mu)
    # print(k,muN[k],muC[k])
  MAAE_N=np.median(muN)
  print("sample size" , N_SAMPLE)
  print("MAAE NN",MAAE_N)
  MAAE_C=np.median(muC)
  print("MAAE CWO",MAAE_C)
  print('loooooolllllll')

# Hyper Parameter Tuning

In [ ]:
!pip install keras-tuner==1.0.0
import kerastuner

In [ ]:
from kerastuner.tuners import RandomSearch
import time
LOG_DIR = f"{int(time.time())}"

In [ ]:
def tempNN(hp):
  Beta2 = tf.keras.models.Sequential()
  Beta2.add(tf.keras.layers.Dense(hp.Int('input_units',
                                         min_value=10,
                                         max_value=100,
                                         step=10), activation='relu',input_shape=(2,)))
  Beta2.add(tf.keras.layers.Dropout(hp.Float('dropout',
                                             min_value=0.0,
                                             max_value=0.4,
                                             default=0.1,
                                             step=0.05,)))
  for i in range(hp.Int('n_layers', min_value=0, max_value=4,
                                         step=1)):
    Beta2.add(tf.keras.layers.Dense(hp.Int(f'conv_{i}_units',
                                           min_value=10,
                                           max_value=100,
                                           step=10), activation='relu'))
    Beta2.add(tf.keras.layers.Dropout(hp.Float(f'dropout_{i}_',
                                             min_value=0.0,
                                             max_value=0.4,
                                             default=0.1,
                                             step=0.05,)))

  Beta2.add(tf.keras.layers.Dense(1, activation='linear'))
  Beta2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate',
                                                                            min_value=1e-5,
                                                                            max_value=1e-2,
                                                                            sampling='LOG',
                                                                            default=1e-3)), loss='mse')

  return Beta2

In [ ]:
tuner = RandomSearch(
    tempNN,
    objective='val_loss',
    max_trials=5,  # how many model variations to test?
    executions_per_trial=4,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

In [ ]:
data=fd.sample(n=5000).reset_index(drop=True)
X = data[['x1','x2']].values.reshape(-1,2)
Y= data['y2'].values.reshape(-1,1)
sampleWeight = weightGenerator(data)




early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
print(X.shape[1])

7


In [ ]:
tuner.search(X,Y,epochs=500,
             validation_split=0.2,
             shuffle = True,
             callbacks=[early_stop],
             verbose=0
             ,sample_weight=sampleWeight
         )

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
hpDict=tuner.get_best_hyperparameters()[0].values

In [ ]:
hpDict={'conv_0_units': 90,
 'conv_1_units': 90,
 'conv_2_units': 50,
 'conv_3_units': 10,
 'dropout': 0.2,
 'dropout_0_': 0.05,
 'dropout_1_': 0.1,
 'dropout_2_': 0.35000000000000003,
 'dropout_3_': 0.05,
 'input_units': 90,
 'learning_rate': 0.001338614394601499,
 'n_layers': 4}

In [ ]:
#for 8
hpDict={'conv_0_units': 50,
 'dropout': 0.05,
 'dropout_0_': 0.15000000000000002,
 'input_units': 40,
 'learning_rate': 0.00503822561891769,
 'n_layers': 1}